In [51]:
import os
import shutil
from multiprocessing import Pool

import librosa
import pandas as pd
import soundfile as sf

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

tqdm.pandas()

lang = 'kn'

In [52]:
data_dir = f"/home/gokulkarthikk/datasets/googletts/{lang}/raw" # update the path
data_dir_new = f"/home/gokulkarthikk/datasets/googletts/{lang}/processed" # update the path

# Process to LJSpeech Format

In [53]:
if not os.path.exists(data_dir_new):
    os.makedirs(data_dir_new)

In [54]:
shutil.copytree(f'{data_dir}/male', f'{data_dir_new}/wavs')

'/home/gokulkarthikk/datasets/googletts/kn/processed/wavs'

In [55]:
shutil.copytree(f'{data_dir}/female', f'{data_dir_new}/wavs', dirs_exist_ok=True)

'/home/gokulkarthikk/datasets/googletts/kn/processed/wavs'

In [56]:
os.remove(f'{data_dir_new}/wavs/LICENSE')
os.remove(f'{data_dir_new}/wavs/line_index.tsv')

In [57]:
metadata_male_fp = f"{data_dir}/male/line_index.tsv"
metadata_male = pd.read_csv(metadata_male_fp, sep='\t', header=None, names=['id', 'text'])
metadata_male['speaker'] = metadata_male['id'].str.rsplit('_', 1).str[0]
metadata_male['gender'] = 'male'
print(metadata_male.shape)
metadata_male.head()

(2214, 4)


,id,text,speaker,gender
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male


In [58]:
metadata_female_fp = f"{data_dir}/female/line_index.tsv"
metadata_female = pd.read_csv(metadata_female_fp, sep='\t', header=None, names=['id', 'text'])
metadata_female['speaker'] = metadata_female['id'].str.rsplit('_', 1).str[0]
metadata_female['gender'] = 'female'
print(metadata_female.shape)
metadata_female.head()

(2186, 4)


,id,text,speaker,gender
0,knf_00557_01097230618,ಪರ್ವತಗಳು ವಾಯುವಿಗನುಗುಣವಾಗಿ ಕಂಪಿಸುವುವು.,knf_00557,female
1,knf_03271_01629528503,ಚಿತ್ರವನ್ನು ಕೆಲವೊಮ್ಮೆ ವಾರ್ನಿಷ್ ಅಥವಾ ಕೆಲವು ಇತರ ಮ...,knf_03271,female
2,knf_03574_00145342156,ಹಚ್ಚುವ ಮೊದಲು ಲೋಹಗಳ ಮೈಯನ್ನು ಉಜ್ಜಿ ಒರಟು ಮಾಡುವುದು...,knf_03574,female
3,knf_00557_01945435360,ಹೂಗೊಂಚಲಿನಲ್ಲಿ ಹೆಣ್ಣು ಹೂಗಳು ಮಾತ್ರವಿದೆ,knf_00557,female
4,knf_02173_02031667415,ನಾಲ್ನೂರು ಗಜಗಳಲ್ಲಿ,knf_02173,female


In [59]:
metadata = pd.concat([metadata_male, metadata_female]).reset_index(drop=True)
metadata

,id,text,speaker,gender
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male
...,...,...,...,...
4395,knf_01493_00549895262,ಮುಂದಿನ ಬೇಸಗೆಯಲ್ಲಿ ಫಸಲು ಬರುತ್ತದೆ,knf_01493,female
4396,knf_01796_01447306308,ಒಂದು ಪೋಷಕವಸ್ತುವಿನ ಕೊರತೆಯಿದ್ದಾಗ,knf_01796,female
4397,knf_08476_00629542551,ಆದರೆ ಅಂತರ್ವ್ಯಾಪ್ತಿಗೆ ಮತ್ತೊಂದು ಅರ್ಥವೂ ಉಂಟು,knf_08476,female
4398,knf_09696_00446271427,ಆಯುರ್ವೇದವನ್ನು ಪ್ರಜಾಪತಿ ದಕ್ಷನಿಗೆ ಬ್ರಹ್ಮ ಉಪದೇಶಿಸಿದ.,knf_09696,female


In [60]:
metadata.to_csv(f'{data_dir_new}/metadata.csv', sep='|', index=False, header=False)

# Resampling

In [61]:
if not os.path.exists(f'{data_dir_new}/wavs-20k'):
    os.makedirs(f'{data_dir_new}/wavs-20k')

In [62]:
def resample_file(func_args):
    fp_src, fp_dst, output_sr = func_args
    y, sr = librosa.load(fp_src, sr=output_sr)
    sf.write(fp_dst, y, sr)

In [63]:
fps_src = [f'{data_dir_new}/wavs/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
fps_dst = [f'{data_dir_new}/wavs-20k/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
srs = [22050] * len(fps_src)

  0%|          | 0/4400 [00:00<?, ?it/s]

  0%|          | 0/4400 [00:00<?, ?it/s]

In [64]:
audio_files = list(zip(fps_src, fps_dst, srs))
with Pool(processes=16) as p:
    with tqdm(total=len(fps_src)) as pbar:
        for i, _ in enumerate(p.imap_unordered(resample_file, audio_files)):
            pbar.update()

  0%|          | 0/4400 [00:00<?, ?it/s]

In [65]:
# audio_files = list(zip(fps_src, fps_dst, srs))
# for audio_file in tqdm(audio_files):
#     resample_file(audio_file)

# Filter data

In [66]:
metadata = pd.read_csv(f'{data_dir_new}/metadata.csv', sep='|', names=['id', 'text', 'speaker', 'gender'])
metadata.head()

,id,text,speaker,gender
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male


In [67]:
metadata['text_length'] = metadata['text'].str.len()
print(len(metadata))
metadata = metadata[~metadata['text_length'].isna()]
metadata['text_length'] = metadata['text_length'].astype(int)
print(len(metadata))
metadata.head()

4400
4400


,id,text,speaker,gender,text_length
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male,34
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male,87
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male,42
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male,75
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male,75


In [68]:
def get_duration(id_):
    fp = f'{data_dir_new}/wavs-20k/{id_}.wav'
    y, sr = librosa.load(fp)
    duration = librosa.get_duration(y=y, sr=sr)
    return duration

In [69]:
metadata['audio_duration'] = metadata['id'].progress_apply(get_duration)
print(len(metadata))
metadata = metadata[metadata['audio_duration']>0]
print(len(metadata))
metadata.head()

  0%|          | 0/4400 [00:00<?, ?it/s]

4400
4400


,id,text,speaker,gender,text_length,audio_duration
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male,34,3.925351
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male,87,8.960000
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male,42,4.608027
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male,75,8.874694
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male,75,6.997370


In [70]:
metadata['audio_duration'].sum()/(60*60)

8.487808528596624

In [71]:
metadata.describe()

,text_length,audio_duration
count,4400.000000,4400.000000
mean,59.937500,6.944571
std,34.133098,3.233102
min,2.000000,1.280000
25%,34.000000,4.522676
50%,56.000000,6.400000
75%,81.000000,8.789342
max,198.000000,26.794694


In [72]:
metadata.to_csv(f'{data_dir_new}/metadata.csv', sep='|', index=False, header=False)

# Make train-test splits

In [73]:
df = pd.read_csv(f'{data_dir_new}/metadata.csv', sep='|', names=['id', 'text', 'speaker', 'gender', 'text_length', 'audio_duration'])
print(df.shape)
df.head()

(4400, 6)


,id,text,speaker,gender,text_length,audio_duration
0,knm_05927_01699170109,ಬಾಗಿಲಿಗೆ ಬಂದ ಜ್ಯೋತಿಯನ್ನು ಸ್ವಾಗತಿಸಿ,knm_05927,male,34,3.925351
1,knm_08025_01815510700,ತೊಡೆಸಂದಿಯ ಅಂಡವಾಯುಗಳು ಪುರುಷರ ಮತ್ತು ಮಹಿಳೆಯರಲ್ಲಿನ...,knm_08025,male,87,8.960000
2,knm_00574_01153125164,ನಿಲ್ದಾಣದ ಮೂಲಕ ಮುಕ್ತವಾಗಿ ತೇಲಾಡುತ್ತ ಮಲಗಬಹುದು,knm_00574,male,42,4.608027
3,knm_01114_01558059071,ಅಂಡಮಾನಿನ ಪುಲಗ ಹಾಗೂ ಪ್ರಾಚೀನ ಭಾರತದ ವರುಣ ಅಂತರಿಕ್ಷ...,knm_01114,male,75,8.874694
4,knm_00271_00413960012,ಈ ಸಮ್ಮೇಳನ ಕೇಂದ್ರ ಬ್ಯಾಂಕುಗಳೊಳಗೆ ಬೆಳೆಯಬೇಕಾದ ಸಹಕಾ...,knm_00271,male,75,6.997370


In [82]:
filtered_speakers = [k for k, v in df['speaker'].value_counts().to_dict().items() if v > 2]
df = df[df['speaker'].isin(filtered_speakers)]
print(df.shape)

(4398, 6)


In [83]:
df_train, df_test = train_test_split(df, test_size=100, stratify=df['speaker'], random_state=0)
print(len(df_train), len(df_test))

4298 100


In [84]:
df_train.to_csv(f'{data_dir_new}/metadata_train.csv', sep='|', index=False, header=False)
df_test.to_csv(f'{data_dir_new}/metadata_test.csv', sep='|', index=False, header=False)